In [1]:
import random
from dataclasses import dataclass
from typing import List, Tuple, Iterable, Sequence

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

Функция make_labels для отыскания пробелов. input - данные с промущенными пробелами, input_with_spaces - правильные данные. На выходе функции строка x_true полностью без пробелов и список y, где y[i]=0 соответсвует отстутвию пробела на i+1 позиции в строке, y[i]=1 соотвествует наличию пробела на i+1 позиции в строке, y[i]=-1 соотвествует наличию пробела в строке, который есть и в input, и в input_with_spaces, его предсказывать не надо.

In [2]:
def norm(s):
    return " ".join((s or "").strip().split())

def make_labels(input, input_with_spaces):
    s  = (input or "")
    s_spaces  = " ".join((input_with_spaces or "").strip().split())
    x_true  = s_spaces.replace(" ", "")
    x  = s.replace(" ", "")
    if x_true != x:
        return "", []
    
    y = []
    for i, ch in enumerate(s_spaces):
        if ch != " ":
            y.append(1 if (i + 1 < len(s_spaces) and s_spaces[i + 1] == " ") else 0)

    k = -1
    for i, ch in enumerate(s):
        if ch != " ":
            k += 1
            if i + 1 < len(s) and s[i + 1] == " ":
                y[k] = -1
    return x_true, y

Класс словаря, в котором записаны все символы, встретившиеся в датасете плюс символ PAD окончания и символ UNK, которым помечаются символы, не входящие в словарь

In [3]:
class CharVocab:
    PAD = "<PAD>"
    UNK = "<UNK>"

    def __init__(self, chars = ()):
        self.itos = [self.PAD, self.UNK]
        self.stoi = {self.PAD: 0, self.UNK: 1}
        self.update(chars)

    def update(self, chars):
        for ch in chars:
            if ch not in self.stoi:
                self.stoi[ch] = len(self.itos)
                self.itos.append(ch)

    def encode(self, s):
        return [self.stoi.get(ch, self.stoi[self.UNK]) for ch in s]

    def __len__(self):
        return len(self.itos)


def build_vocab_from_pairs(pairs):
    chars = []
    for _, with_spaces in pairs:
        x = norm(with_spaces).replace(" ", "")
        chars.extend(list(x))
    return CharVocab(chars)

Класс для удобства работы с данными

In [4]:
@dataclass
class Sample:
    ids: List[int]
    labels: List[int]

class PairDataset(Dataset):
    def __init__(self, pairs, vocab):
        self.vocab = vocab
        self.samples = []
        for input_text, with_spaces in pairs:
            x, y = make_labels(input_text, with_spaces)
            if len(x) < 2 or not y:
                continue
            self.samples.append(Sample(ids=vocab.encode(x), labels=y))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

In [5]:
def collate(batch, pad_id = 0):
    T = max(len(s.ids) for s in batch)
    B = len(batch)
    x = torch.full((B, T), pad_id, dtype=torch.long)
    y = torch.full((B, T), -1.0, dtype=torch.float)
    lengths = torch.zeros(B, dtype=torch.long)
    for i, s in enumerate(batch):
        n = len(s.ids)
        x[i, :n] = torch.tensor(s.ids, dtype=torch.long)
        y[i, :n] = torch.tensor(s.labels, dtype=torch.float)
        lengths[i] = n
    return x, y, lengths

В качестве основы была выбрана LSTM. Для борьбы с переобучением используется dropout. На выходе модель выдает логиты

In [6]:
class BiLSTMSpace(nn.Module):
    def __init__(self, vocab_size, emb_dim = 64, hidden = 128, num_layers = 2, dropout = 0.2):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.lstm = nn.LSTM(emb_dim, hidden, num_layers=num_layers, batch_first=True, bidirectional=True, dropout=dropout if num_layers > 1 else 0.0)
        self.linear = nn.Linear(hidden * 2, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, lengths):
        emb = self.dropout(self.emb(x))
        packed = pack_padded_sequence(emb, lengths.cpu(), batch_first=True, enforce_sorted=False)
        out, _ = self.lstm(packed)
        out, _ = pad_packed_sequence(out, batch_first=True)
        logits = self.linear(self.dropout(out)).squeeze(-1)
        return logits

Функция masked_bce_with_logits представляет собой кросс-энтропию в качестве лосс-функции, где не учитываются элементы с y[i]=-1, то есть которые не надо предсказывать и, следовательно, штрафовать

In [7]:
def masked_bce_with_logits(logits, labels):
    B, T = logits.shape
    valid = (labels != -1.0).float()
    if T > 0:
        valid[:, -1] = 0.0
    loss_fn = nn.BCEWithLogitsLoss(reduction="none")
    loss = (loss_fn(logits, torch.clamp(labels, 0.0, 1.0)) * valid).sum() / valid.sum().clamp(min=1.0)
    return loss

Процесс обучения стандартный. Для контроля за процессом обучения выводится количество пройденных эпох и средний лосс.

In [8]:
def train_model_simple(train_pairs, emb=64, hidden=128, layers=2, dropout=0.2, batch_size=64, epochs=10, lr=1e-3, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    vocab = build_vocab_from_pairs(train_pairs)
    train_ds = PairDataset(train_pairs, vocab)
    coll = lambda b: collate(b, pad_id=0)
    g = torch.Generator()
    g.manual_seed(SEED)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, generator=g, num_workers=0, collate_fn=coll)

    model = BiLSTMSpace(len(vocab), emb_dim=emb, hidden=hidden, num_layers=layers, dropout=dropout).to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=lr)


    for ep in range(1, epochs + 1):
        model.train()
        run = 0.0
        for x, y, lengths in train_loader:
            x, y, lengths = x.to(device), y.to(device), lengths.to(device)
            logits = model(x, lengths)
            loss = masked_bce_with_logits(logits, y)
            opt.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
            run += loss.item() * x.size(0)
        tr_loss = run / max(1, len(train_loader.dataset))
        print(f"Epoch {ep:02d} | train_loss={tr_loss:.4f}")

    return model, vocab

Функция предсказания возвращает строку с расставленными пробелами, список с позициями добавленных пробелов и вероятности.

In [9]:
@torch.no_grad()
def predict_string(model, vocab, text, threshold = 0.5, device=None):
    if device is None:
        device = next(model.parameters()).device

    s = text
    compact_chars, existing_space = [], []
    for i, ch in enumerate(s):
        if ch != " ":
            compact_chars.append(ch)
            existing_space.append(i + 1 < len(s) and s[i + 1] == " ")

    x = "".join(compact_chars)
    if len(x) <= 1:
        return s, [], [0.0] * len(x)

    ids = torch.tensor([[vocab.stoi.get(ch, vocab.stoi[CharVocab.UNK]) for ch in x]], dtype=torch.long, device=device)
    lengths = torch.tensor([ids.size(1)], dtype=torch.long, device=device)
    logits = model(ids, lengths)
    probs = torch.sigmoid(logits)[0].tolist()

    positions = [i for i, p in enumerate(probs[:len(x)-1]) if p >= threshold and not existing_space[i]]

    out = []
    k = -1
    for i, ch in enumerate(s):
        out.append(ch)
        if ch != " ":
            k += 1
            if not existing_space[k] and k in positions:
                out.append(" ")
    return "".join(out), positions, probs[:len(x)]

In [10]:
def f1_macro_from_texts(inputs, gts, preds):
    f1_sum, n = 0.0, 0
    for s_in, s_gt, s_pred in zip(inputs, gts, preds):
        x_true, y_true = make_labels(s_in, s_gt)
        if not y_true: 
            continue
        L = len(y_true)
        valid = [v != -1 for v in y_true]
        if L > 0:
            valid[-1] = False

        _, y_pred = make_labels(s_in, s_pred)

        pred_set = {i for i, p in enumerate(y_pred[:L]) if p == 1 and valid[i]}
        gold_set = {i for i, g in enumerate(y_true[:L]) if g == 1 and valid[i]}

        inter = len(pred_set & gold_set)
        prec = inter / len(pred_set) if pred_set else (1.0 if not gold_set else 0.0)
        rec  = inter / len(gold_set) if gold_set else (1.0 if not pred_set else 0.0)
        f1   = 0.0 if (prec + rec) == 0 else 2 * prec * rec / (prec + rec)
        f1_sum += f1; n += 1
    return f1_sum / max(1, n)

In [11]:
@torch.no_grad()
def evaluate_and_predict(model, vocab, pairs, threshold=0.5, batch_size=64):
    device = next(model.parameters()).device

    inputs = [a for a, _ in pairs]
    true_strings    = [b for _, b in pairs]
    restored_list = []
    positions_list = []
    for s_in in inputs:
        restored, positions, _  = predict_string(model, vocab, s_in, threshold=threshold, device=device)
        restored_list.append(restored)
        positions_list.append(positions)
    f1 = f1_macro_from_texts(inputs, true_strings, restored_list)

    return restored_list, positions_list, f1

In [ ]:
TRAIN_CSV = 'avito_train1.csv'

train_df = pd.read_csv(TRAIN_CSV)
train_pairs = list(train_df[["no_spaces","with_spaces"]].astype(str).itertuples(index=False, name=None))

model, vocab = train_model_simple(train_pairs, epochs=25, batch_size=64, lr=1e-3)

Epoch 01 | train_loss=0.4211
Epoch 02 | train_loss=0.2384
Epoch 03 | train_loss=0.2001
Epoch 04 | train_loss=0.1800
Epoch 05 | train_loss=0.1655
Epoch 06 | train_loss=0.1549
Epoch 07 | train_loss=0.1467
Epoch 08 | train_loss=0.1399
Epoch 09 | train_loss=0.1341
Epoch 10 | train_loss=0.1292


In [ ]:
path = "dataset_1937770_3.txt"

tmp = pd.read_fwf(path, header=0, names=["raw"], encoding="utf-8")

task_data = tmp["raw"].str.split(",", n=1, expand=True)
task_data.columns = ["id", "text_no_spaces"]
task_data["id"] = task_data["id"].astype(int)

In [16]:
restored_list = []
positions_list = []
for s_in in task_data['text_no_spaces']:
    s_in = str(s_in) 
    restored, positions, _  = predict_string(model, vocab, s_in, threshold=0.5)
    restored_list.append(restored)
    positions_list.append(positions)

In [278]:
task_data['predicted_positions'] = positions_list

In [ ]:
task_data['predicted_positions'] = task_data['predicted_positions'].astype(str)
submission = task_data[['id', 'predicted_positions']]
submission.to_csv("submission.csv", index=False, encoding="utf-8")

In [17]:
print(restored_list)

['куплю айфон 14про', 'ищу дом в Подмосковье', 'сдаю квартиру с мебелью итехникой', 'новый дивандоставка недорого', 'отдам даромкошку', 'работа в Москвеу даленно', 'куплю телевизор Philips', 'ищу грузчиковдляпереезда', 'ремонтк вартир подключ', 'куплю ноутбук HP', 'ищу квартиру у метро', 'новая микроволновка Samsung', 'срочно продам велосипед', 'куплю гитаруFender', 'ищу репетитор апобиологии', 'сдаю гараж надли тельный срок', 'куплю диванбу', 'ищу мастера по ремонту холодильников', 'новыйшкаф доставка сегодня', 'куплю Xbox One', 'ищу подработку повечерам', 'сдам комнату сту дентке', 'куплю старую книгу', 'ищу собак улабрадор', 'новый телефон Xiaomi 13', 'куплю Playstation 5 диск', 'ищу комнату в центре города', 'срочно нужнаняняре бенку', 'куплю стиральную машину Indesit', 'ищу детскуюкроватку бу', 'новаякуртка доставка', 'куплю велосипед Mer ida', 'ищу врачаофталь молога', 'сдаю квартиру в центре Москвы', 'куплю холодильник Samsung', 'ищу кошку британскую', 'новый ноутбук доставка за